# Advent of Code 2021

This solution (Jupyter notebook; coconut 1.5.0 on python 3.7.11) by kannix68, @ 2021-12.  \
Using anaconda distro, conda v4.10.3, and coconut language. installation on MacOS v10.14.6 "Mojave".

Reddit Advent of Code [solution_megathreads - adventofcode](https://www.reddit.com/r/adventofcode/wiki/solution_megathreads#wiki_december_2021)

In [ ]:
import copy
import itertools
import logging
import re
import sys
import time

from collections import defaultdict

import numpy as np
import pandas as pd

import pylib.aochelper as aoc
#from pylib.aochelper import map_list as mapl
#from pylib.aochelper import filter_list as filterl

f"Python version: {sys.version}" |> print
f"Version info: {sys.version_info}" |> print

log = aoc.getLogger(__name__)
f"Initial log-level={aoc.getLogLevelName(log.getEffectiveLevel())}." |> print

## Problem domain code

### Day 1: Sonar Sweep

In [ ]:
"Day 1" |> print

tests = """
199
200
208
210
200
207
240
269
260
263""".strip()

In [ ]:
def solve_d01pt1(inp):
  """Solve Day 1 part 1."""
  inp = inp |> .split() |> map$(int)
  outp = pd.Series(inp).diff()[1:].astype(int).tolist()
  outp = outp |> filter$(it -> it > 0)
  return outp |> list |> len

In [ ]:
expected = 7
result = solve_d01pt1(tests)
aoc.assert_msg("test solve day 1 part 1", result == expected) 

In [ ]:
ins = aoc.read_file_to_str("./in/day01.in")
out = solve_d01pt1(ins)
f"day 1 part 1 output: {out}" |> print

In [ ]:
"Day 1 part 2" |> print

In [ ]:
def solve_d01pt2(inp):
  """Solve Day 1 part 2."""
  inp = inp |> .split() |> map$(int)
  outp = pd.Series(inp).rolling(3).sum().diff()[3:].astype(int).tolist()
  return outp |> filter$(-> _ > 0) |> list |> len

In [ ]:
expected = 5
result = solve_d01pt2(tests)
aoc.assert_msg("test solve day 1 part 2", result == expected) 

In [ ]:
out = solve_d01pt2(ins)
f"day 1 part 2 output: {out}" |> print

## Day 2: Dive!

In [ ]:
"Day 2" |> print

In [ ]:
def iterate(pos, cmd):
  HPOS, DEPTH = 0, 1
  direct, val = cmd.split(" ")
  val = int(val)
  pos = pos.copy()
  case direct:
    match "forward":
      pos[HPOS] += val
    match "down":
      pos[DEPTH] += val
    match "up":
      pos[DEPTH] -= val
  return pos

def solve_d02pt1(inp):
  pos = [0, 0]
  for cmd in inp.splitlines():
    pos = iterate(pos, cmd)
  return pos |> reduce$(*)

tests = """
forward 5
down 5
forward 8
up 3
down 8
forward 2""".strip()

expected = 150
result = solve_d02pt1(tests)
aoc.assert_msg("test solve day 1 part 1", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day02.in")
out = solve_d02pt1(ins)
f"day 2 part 1 output: {out}" |> print

In [ ]:
"Day 2 part 2" |> print

In [ ]:
def iterate2(pos, cmd):
  HPOS, DEPTH, AIM = 0, 1, 2
  direct, val = cmd.split(" ")
  val = int(val)
  pos = pos.copy()
  case direct:
    match "forward":
      pos[HPOS] += val
      pos[DEPTH] += pos[AIM] * val
    match "down":
      pos[AIM] += val
    match "up":
      pos[AIM] -= val
  return pos

def solve_d02pt2(inp):
  pos = [0, 0, 0]
  for cmd in inp.splitlines():
    pos = iterate2(pos, cmd)
  return pos[0:2] |> reduce$(*)

expected = 900
result = solve_d02pt2(tests)
aoc.assert_msg("test solve day 1 part 2", result == expected)

In [ ]:
# `ins` remains the same
out = solve_d02pt2(ins)
f"day 2 part 2 output: {out}" |> print

## Day 3: Binary Diagnostic

In [ ]:
"Day 3" |> print

In [ ]:
tests = """
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
""".strip()

def T(lst):
  """Transpose a 2d list."""
  return [list(i) for i in zip(*lst)]  # transpose

def bitlist2int(bitlist):
  return int(str.join('', map(str, bitlist)), 2)

def solve_d03pt1(ins):
  lst = ins.splitlines()
  for idx in range(len(lst)):
    lst[idx] = map(int, list(lst[idx])) |> list
  l = len(lst)
  log.debug(lst)
  in_t = T(lst)
  log.debug(in_t)
  blst = []
  cblst = []
  for col in in_t:
    c0, c1 = col.count(0), l - col.count(0)
    if c0 > c1:
      i = 0
      c = 1
    else:
      i = 1
      c = 0
    blst.append(i)
    cblst.append(c)
  log.debug(blst)
  bnum = bitlist2int(blst)
  cnum = bitlist2int(cblst)
  log.debug([bnum, cnum])
  return bnum * cnum

expected = 198
log.setLevel(logging.INFO)
result = solve_d03pt1(tests)
aoc.assert_msg("test solve day 3 part 1", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day03.in")
out = solve_d03pt1(ins)
f"day 3 part 1 output: {out}" |> print

In [ ]:
"Day 3 part 2" |> print

In [ ]:
def solve_d03pt2(ins):
  lst = ins.splitlines()
  for idx in range(len(lst)):
    lst[idx] = map(int, list(lst[idx])) |> list
  elemlen = len(lst[0])
  lstlen = len(lst)
  tlst = T(lst)
  log.debug(lst)
  log.info(f"elemlen={elemlen}, lstlen={lstlen}")
  olst = copy.deepcopy(lst)
  for i in range(elemlen):
    sm1 = tlst[i].count(1)
    sm0 = lstlen - sm1
    if sm1 >= sm0:
      trg = 1
    else:
      trg = 0
    log.debug(f"oxy iter-#{i} lstlen={lstlen} trg={trg}")
    lst = lst |> filter$(-> _[i] == trg) |> list
    tlst = T(lst)
    lstlen = len(lst)
    if lstlen <= 1:
      oxyrat_lst = lst[0]
      oxyrat = bitlist2int(oxyrat_lst)
      log.debug(f"oxyrat={oxyrat_lst} => {oxyrat}")
      break
  lst = olst
  tlst = T(olst)
  lstlen = len(lst)
  for i in range(elemlen):
    sm1 = tlst[i].count(1)
    sm0 = lstlen - sm1
    if sm1 >= sm0:
      trg = 0
    else:
      trg = 1
    log.debug(f"co2 iter-#{i} lstlen={lstlen} trg={trg}")
    lst = lst |> filter$(-> _[i] == trg) |> list
    tlst = T(lst)
    lstlen = len(lst)
    if lstlen <= 1:
      co2rat_lst = lst[0]
      co2rat = bitlist2int(co2rat_lst)
      log.debug(f"co2rat={co2rat_lst} => {co2rat}")
      break
  return oxyrat * co2rat

expected = 230
log.setLevel(logging.INFO)
result = solve_d03pt2(tests)
aoc.assert_msg("test solve day 3 part 2", result == expected)

In [ ]:
out = solve_d03pt2(ins)
f"day 3 part 2 output: {out}" |> print

## Day 4: Giant Squid

In [ ]:
print("Day 4, bingo!")

In [ ]:
tests = """
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
""".strip()

In [ ]:
"Day 4 TBD, see python solution" |> print

## Day 5: Hydrothermal Venture

In [ ]:
"Day 5" |> print

In [ ]:
tests = """
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
""".strip()

In [ ]:
def solve_d05pt1(ins):
  lines = ins.splitlines() |> map$(-> _.split(" -> ")) |> list
  for lnidx in range(len(lines)):
    for ptidx in range(2):
      lines[lnidx][ptidx] = lines[lnidx][ptidx].split(",") |> map$(int) |> list
  log.trace(lines)
  grid = defaultdict(int)
  for line in lines:
    ptst, pten = line
    if ptst[0] == pten[0]:  # horiz
      log.trace(f"horiz line {line}")
      x = ptst[0]
      for y in range(min(ptst[1], pten[1]), max(ptst[1], pten[1])+1):
        grid[tuple([x,y])] += 1
    elif ptst[1] == pten[1]:  # vert
      log.trace(f"vert line {line}")
      y = ptst[1]
      for x in range(min(ptst[0], pten[0]), max(ptst[0], pten[0])+1):
        grid[tuple([x,y])] += 1
    else:
      pass
  log.trace(grid)
  intersects = grid.values() |> filter$(-> _ > 1) |> list |> len
  log.debug(f"intersects-#={intersects}")
  return intersects

expected = 5
log.setLevel(logging.INFO)
result = solve_d05pt1(tests)
aoc.assert_msg("test solve day 5 part 1", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day05.in")
out = solve_d05pt1(ins)
f"day 5 part 1 output: {out}" |> print

In [ ]:
"Day 5 part 2" |> print

In [ ]:
def scan_grid(lines):
  grid = defaultdict(int)
  for line in lines:
    ptst, pten = line
    if ptst[0] == pten[0] and ptst[1] == pten[1]:  # point
      grid[tuple(ptst)] += 1
    elif ptst[0] == pten[0]:  # horiz
      log.trace(f"horiz line {line}")
      x = ptst[0]
      for y in range(min(ptst[1], pten[1]), max(ptst[1], pten[1])+1):
        grid[tuple([x,y])] += 1
    elif ptst[1] == pten[1]:  # vert
      log.trace(f"vert line {line}")
      y = ptst[1]
      for x in range(min(ptst[0], pten[0]), max(ptst[0], pten[0])+1):
        grid[tuple([x,y])] += 1
    else:  # diagonal
      line = sorted(line, key=lambda it: it[0])
      ptst, pten = line
      if ptst[1] > pten[1]:
        ystp = -1
      else:
        ystp = 1
      log.trace(f"diag line {line}")
      y = ptst[1]
      for x in range(ptst[0], pten[0]+1):
        log.trace(f"set diag line-pt {[x,y]}") 
        grid[tuple([x,y])] += 1
        y += ystp
  log.trace(grid)
  return grid
  
def solve_d05pt2(ins):
  lines = ins.splitlines() |> map$(-> _.split(" -> ")) |> list
  for lnidx in range(len(lines)):
    for ptidx in range(2):
      lines[lnidx][ptidx] = lines[lnidx][ptidx].split(",") |> map$(int) |> list
  log.trace(lines)
  grid = scan_grid(lines)
  intersects = grid.values() |> filter$(-> _ > 1) |> list |> len
  log.debug(f"intersects-#={intersects}")
  #log.debug(f"grid={grid}")
  return intersects

expected = 12
log.setLevel(logging.INFO)
result = solve_d05pt2(tests)
aoc.assert_msg("test solve day 5 part 2", result == expected)

In [ ]:
out = solve_d05pt2(ins)
f"day 5 part 2 output: {out}" |> print

## Day 6: Lanternfish
DE: [Nikolaus von Myra](https://de.wikipedia.org/wiki/Nikolaus_von_Myra)  \
EN: [Saint Nicholas](https://en.wikipedia.org/wiki/Saint_Nicholas)  \
ES: [Nicolás de Bari](https://es.wikipedia.org/wiki/Nicol%C3%A1s_de_Bari)

In [ ]:
"Day 6" |> print

In [ ]:
tests = "3,4,3,1,2"

In [ ]:
def solve_d06pt1(ins, n):
  """Lanternfish, "brute force"."""
  cells = ins.split(",") |> map$(int) |> list
  log.trace(f"lfish={cells}")
  log.debug(f"lfish-num={len(cells)}")
  tms = time.time()
  for itr in range(n):
    log.debug(f"iter-#{itr} took {round(time.time()-tms, 2)}s len={len(cells)}")
    toappend = 0
    for idx in range(len(cells)):
      case cells[idx]:
        match 0:
          toappend += 1
          cells[idx] = 6
      else:
        cells[idx] -= 1
    cells = cells + [8] * toappend
    log.trace(f"iter#{itr} lfish={cells}")
  log.info(f"End iter-#{itr} took {round(time.time()-tms, 2)}s len={len(cells)}")
  return len(cells)

expected = [[18, 26], [80,5934]]
log.setLevel(logging.INFO)
for e in expected:
  n, expct = e
  result = solve_d06pt1(tests, n)
  aoc.assert_msg(f"test solve day 6 part 1; expect fish={expct} <= n={n} found {result}", result == expct)

In [ ]:
ins = aoc.read_file_to_str("./in/day06.in")
out = solve_d06pt1(ins, 80)
f"day 6 part 1 output: {out}" |> print

In [ ]:
"Day 6 part 2" |> print

In [ ]:
def solve_d06pt2(ins, n):
  """Lanternfish, just keep counting cellstates-number."""
  tms = time.time()
  cells = ins.split(",") |> map$(int) |> list
  log.debug(f"lfish-num={len(cells)}")
  cellsts = []
  for idx in range(9):
    cellsts.append(cells.count(idx))
  log.debug(f"initial cellstates={cellsts}")
  for itr in range(n):
    toappend, cellsts = cellsts[0], cellsts[1:]
    assert len(cellsts) == 8, f"len={len(cellsts)}"
    cellsts[6] += toappend
    cellsts = cellsts + [toappend]
    if itr % 10 == 0:
      #log.info(f"iter-#{itr} count={sum(cellsts)}. cellstates={cellsts}. took {int(time.time()-tms)}s")
      log.debug(f"iter-#{itr} count={sum(cellsts)}. took {int(time.time()-tms)}s")
  log.info(f"iter-#{itr} ends. took {round(time.time()-tms, 4)}s")
  return sum(cellsts)

In [ ]:
expected = [[256, 26984457539]]
log.setLevel(logging.INFO)
for e in expected:
  n, expct = e
  result = solve_d06pt2(tests, n)
  aoc.assert_msg(f"test solve day 6 part 1; expect fish={expct} <= n={n} found {result}", result == expct)

In [ ]:
out = solve_d06pt2(ins, 256)
f"day 6 part 2 output: {out}" |> print

## Day 7: The Treachery of Whales

In [ ]:
"Day 7" |> print

In [ ]:
tests = "16,1,2,0,4,2,7,1,2,14"

In [ ]:
def solve_d07pt1(ins):
  """Day 7 pt1 solution."""
  ins = ins.split(",") |> map$(int) |> list
  mn, mx = min(ins), max(ins)
  fc_min = 1e20
  for trg in range(mn, mx+1):
    fc = 0
    for xpos in ins:
      fc += abs(xpos - trg)
    if fc < fc_min:
      log.debug(f"pos={trg}, new min fuelcost={fc}")
      fc_min = fc
  return fc_min

expected = 37
log.setLevel(logging.INFO)
result = solve_d07pt1(tests)
aoc.assert_msg(f"test solve day 7 part 1; expected={expct}, found {result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day07.in")
out = solve_d07pt1(ins)
f"day 7 part 1 output: {out}" |> print

In [ ]:
def solve_d07pt2(ins):
  """Day 7 pt2 solution."""
  ins = ins.split(",") |> map$(int) |> list
  mn, mx = min(ins), max(ins)
  fc_min = 1e20
  cst = 0
  seq = []
  for idx in range(0, mx+1): # costs per step
    seq.append(cst)
    cst += 1
  costseq = []
  for idx in range(0, mx+1): # costs for distance
    #log.debug(f"dst={idx} costseq={seq[0:idx+1]}")
    costseq.append(sum(seq[0:idx+1]))
  log.debug(seq)
  for trg in range(mn, mx+1):
    fc = 0
    fclst = []
    for xpos in ins:
      dist = abs(xpos - trg)
      fclst.append(costseq[dist])
      fc += costseq[dist]
    log.debug(f"  pos={trg}, fuelcosts={fclst}")
    if fc < fc_min:
      log.debug(f"pos={trg}, new min fuelcost={fc}")
      fc_min = fc
  return fc_min

expected = 168
log.setLevel(logging.INFO)
result = solve_d07pt2(tests)
aoc.assert_msg(f"test solve day 7 part 2; expected={expected}, found {result}", result == expected)

In [ ]:
out = solve_d07pt2(ins)
f"day 7 part 2 output: {out}" |> print

## Day 8: Seven Segment Search

In [ ]:
"Day 8" |> print

In [ ]:
segments = """ 
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
""".strip()
 
d = {
  0: ["a","b","c","e","f","g"],
  1: ["c","f"],
  2: ["a","c","d","e","g"],
  3: ["a","c","d","f","g"],
  4: ["b","c","d","f"],
  5: ["a","b","d","f","g"],
  6: ["a","b","d","e","f","g"],
  7: ["a","c","f"],
  8: ["a","b","c","d","e","f","g"],
  9: ["a","b","c","d","f","g"],
}

d2 = {
  1: ["c","f"],
  7: ["a","c","f"],
  4: ["b","c","d","f"],
  2: ["a","c","d","e","g"],
  3: ["a","c","d","f","g"],
  5: ["a","b","d","f","g"],
  0: ["a","b","c","e","f","g"],
  6: ["a","b","d","e","f","g"],
  9: ["a","b","c","d","f","g"],
  8: ["a","b","c","d","e","f","g"],
}

In [ ]:
tests1 = """acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf""".strip()

tests = """
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
""".strip()

In [ ]:
def solve_d08pt1(ins):
  """Day 8 pt1 solution."""
  ins = ins.splitlines() # |> filter$(-> "|" not in _) |> list
  for idx in range(len(ins)):
    ins[idx] = ins[idx].split(" | ")[1]
  ct = 0
  for ln in ins:
    for grp in ln.split(" "):
      if len(grp) in [2, 4, 3, 7]:
        ct += 1
  log.trace([ins, ct])
  return ct

expected = 26
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
result = solve_d08pt1(tests)
aoc.assert_msg(f"test solve day 8 part 1; expected={expected}, found {result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day08.in")
out = solve_d08pt1(ins)
f"day 8 part 1 output: {out}" |> print

In [ ]:
def solve_d08pt2(ins):
  """Day 8 pt2 solution."""
  return None

expected1 = 5353
expected = 61229
#log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
#result = solve_d08pt2(tests1)
#result = solve_d08pt2(tests)
#aoc.assert_msg(f"test solve day 7 part 2; expected={expected}, found {result}", result == expected)
"Day 8 pt2 solution - TBD." |> print

## Day 9: Smoke Basin

In [ ]:
"Day 9" |> print

tests = """
2199943210
3987894921
9856789892
8767896789
9899965678
""".strip()

In [ ]:
def solve_d09pt1(ins):
  """Day 9 pt1 solution."""
  ins = ins.splitlines() |> map$(-> list(_)) |> list
  xdim , ydim = len(ins[0]), len(ins)
  for y in range(ydim):
    for x in range(xdim):
      ins[y][x] = int(ins[y][x])
  log.debug(f"grid[{xdim},{ydim}]...!")
  log.trace(f"grid[{xdim},{ydim}]")
  locminsum = 0
  for y in range(ydim):
    for x in range(xdim):
      pt = ins[y][x]
      nbs = aoc.get_neighbors(ins, x, y) |> filter$(-> _ != "") |> list
      #log.trace(f"pt[{x},{y}]={pt} neibs={nbs}")
      #all(flag == 0 for (_, _, flag) in items)
      if pt < min(nbs):
        log.trace(f"pt[{x},{y}]={pt} IS locmin, neibs={nbs}")
        locminsum += pt+1
  log.info(f"rc locminsum={locminsum}")
  return locminsum

expected = 15
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
result = solve_d09pt1(tests)
aoc.assert_msg(f"test solve day 8 part 1; expected={expected}, found {result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day09.in")
out = solve_d09pt1(ins)
f"day 9 part 1 output: {out}" |> print

In [ ]:
"Day 9 part 2" |> print

def get_locmins(ins):
  xdim , ydim = len(ins[0]), len(ins)
  locmins = []
  for y in range(ydim):
    for x in range(xdim):
      pt = ins[y][x]
      nbs = aoc.get_neighbors(ins, x, y) |> filter$(-> _ != "") |> list
      #log.trace(f"pt[{x},{y}]={pt} neibs={nbs}")
      if pt != "" and pt < min(nbs):
        log.trace(f"pt[{x},{y}]={pt} IS locmin, neibs={nbs}")
        locmins.append( tuple([x,y]) )
  log.debug(f"rc locmins={locmins}")
  return locmins

def get_neib_coords(strlist: List[str], x: int, y: int) -> List[str]:
  """Get a list of 4 neighbors of cell. May return '' as element if boundary."""
  max_y_idx = len(strlist) - 1
  max_x_idx = len(strlist[0]) - 1
  nbs = []
  if y > 0:
    nbs.append(tuple([x, y-1]))
  if x > 0:
    nbs.append(tuple([x-1, y]))
  if y < max_y_idx:
    nbs.append(tuple([x, y+1]))
  if x < max_x_idx:
    nbs.append(tuple([x+1, y]))
  return nbs

def solve_d09pt2(ins):
  """Day 9 pt1 solution."""
  ins = ins.splitlines() |> map$(-> list(_)) |> list
  xdim , ydim = len(ins[0]), len(ins)
  for y in range(ydim):
    for x in range(xdim):
      val = int(ins[y][x])
      if val < 9:
        ins[y][x] = val
      else:
        ins[y][x] = 9
  log.debug(f"grid[{xdim},{ydim}]...!")
  log.trace(f"grid[{xdim},{ydim}]")
  basinsize_lst = []
  for lmin in get_locmins(ins):
    basin = set()
    to_scan = [lmin]
    minval = ins[lmin[1]][lmin[0]]
    while(len(to_scan)>0):
      pt = to_scan[0]
      basin.add(pt)
      to_scan = to_scan[1:]
      x,y = pt[0], pt[1]
      ptval = ins[y][x]
      nb_coords = get_neib_coords(ins, x, y)
      nb_coords = nb_coords |> filter$(-> _ not in basin and ins[_[1]][_[0]] != "" and ins[_[1]][_[0]] < 9) |> list
      nb_coords = nb_coords |> filter$(-> _ not in to_scan) |> list
      to_scan += nb_coords
      log.trace(f"trg-pt[{x},{y}]={ptval} len(toscan)={len(to_scan)}, nb-coords={nb_coords}")
    log.debug(f"lmin-pt[{lmin}]={minval} basin-len={len(basin)}")
    basinsize_lst.append(len(basin))
  basinsize_lst = sorted(basinsize_lst)
  log.debug(f"basinsize_lst={basinsize_lst}, {basinsize_lst[-3:]}")
  return reduce((*),basinsize_lst[-3:])

expected = 1134
#log.setLevel(logging.DEBUG)
log.setLevel(logging.INFO)
result = solve_d09pt2(tests)
aoc.assert_msg(f"test solve day 8 part 1; expected={expected}, found {result}", result == expected)

In [ ]:
out = solve_d09pt2(ins)
f"day 9 part 2 output: {out}" |> print

## Day 10: Syntax Scoring

In [ ]:
"Day 10" |> print

tests = """
[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]
""".strip()

In [ ]:
def solve_d10pt1(ins):
  """Day 10 pt1 solution."""
  minis = ["()", "[]", "{}", "<>"]
  closing = [")", "]", "}", ">"]
  scores = {")":3, "]":57, "}":1197, ">":25137}
  ins = ins.splitlines()
  score_sum = 0
  for ln in ins:
    #log.debug(f"ln={ln}")
    while any(s in ln for s in minis):
      ln = ln.replace("()", "").replace("[]", "").replace("{}", "").replace("<>", "")
    #log.debug(f"  reduced={ln}")
    if not any(s in ln for s in closing):
      #log.debug("  is ok/open ln")
      continue
    ln2 = ln
    ln2 = ln.replace("(", "").replace("[", "").replace("{", "").replace("<", "")
    breaking = ln2[0]
    score = scores[breaking]
    #log.debug(f"  breaks with score={score}")
    score_sum += score
  return score_sum

expected = 26397
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
result = solve_d10pt1(tests)
aoc.assert_msg(f"test solve day 8 part 1; expected={expected}, found {result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day10.in")
out = solve_d10pt1(ins)
f"day 10 part 1 output: {out}" |> print

In [ ]:
def solve_d10pt2(ins):
  """Day 10 pt1 solution."""
  minis = ["()", "[]", "{}", "<>"]
  closing = [")", "]", "}", ">"]
  scores2 = {")":1, "]":2, "}":3, ">":4}
  matching = {"(": ")", "[": "]", "{": "}", "<": ">" }
  ins = ins.splitlines()
  score_sum = 0
  rcscores = []
  for ln in ins:
    #log.debug(f"ln={ln}")
    while any(s in ln for s in minis):
      ln = ln.replace("()", "").replace("[]", "").replace("{}", "").replace("<>", "")
    #log.debug(f"  reduced={ln}")
    if any(s in ln for s in closing):
      #log.debug("  is corrupt")
      continue
    score = 0
    while len(ln) > 0:
      c = ln[-1]
      ln = ln[:-1]
      score = score*5 + scores2[matching[c]]
      #log.trace(f"red-line={ln}, tmp-score={score}")
    #log.debug(f" score={score} for line")
    rcscores.append( score )
  l = len(rcscores)
  idx = l // 2
  rc = sorted(rcscores)[idx]
  #log.debug(f"rcscores rc={rc}, len={l}, median-idx={idx}, idx ar={rcscores}")
  return rc

expected = 288957
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
result = solve_d10pt2(tests)
aoc.assert_msg(f"test solve day 8 part 1; expected={expected}, found {result}", result == expected)

In [ ]:
out = solve_d10pt2(ins)
f"day 10 part 2 output: {out}" |> print

## Day 11: Dumbo Octopus

In [ ]:
"Day 11" |> print

tests1 = """
11111
19991
19191
19991
11111
""".strip()

tests = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
""".strip()

In [ ]:
class Grid:
  """Grid as 2d list-of-list of chars"""
  def __init__(self, los):
    self.grid = los |> map$(list) |>list
    self.dim = [len(los[0]), len(los)]
    self.iter = 0
    self.md = {}
    self.init_nb_tns()
  
  def __repr__(self):
    return f"Grid[{self.dim}] iter={self.iter} md={self.md}"
  
  def show(self):
    #aoc.represent_strlist(self.grid)
    los = self.grid |> map$(-> str.join("", _))
    return "grid-show:\n" + str.join("\n", los)
  
  def init_nb_tns(self):
    """Diagonal transformations."""
    lst = []
    for x in range(-1, 2):
      for y in range(-1, 2):
        if (x,y) == (0,0):
          continue
        else:
          lst.append( (x,y) )
    self.nb_tns = lst


In [ ]:
def raise1(g, coord):
  x,y = coord[0], coord[1]
  gg = g.grid
  v = gg[y][x]
  if v == "a":
    return False
  else:
    i = int(v)
    if i == 9:
      gg[y][x] = "a"
      return True
    else:
      gg[y][x] = str(i+1)
      return False

def raise_neibs_en(g, coord):
  neibs = g.nb_tns |> map$(->(_[0]+coord[0], _[1]+coord[1])) |> list
  neibs = neibs |> filter$(-> _[0] >= 0 and _[0] < g.dim[0]) |> list
  neibs = neibs |> filter$(-> _[1] >= 0 and _[1] < g.dim[1]) |> list
  log.debug(f"neibs={neibs}")
  new_to_flash = []
  for neib in neibs:
    if raise1(g, neib):
      log.debug(f"raise_neibs_en: new2flash={neib}")
      new_to_flash(neib)
  # TODO: neu geflashte zurückliefern
  return new_to_flash

def raise_en(g):
  gg = g.grid
  flashed = []
  flashs_num = 0
  init = True
  while(len(flashed)>0 or init):
    if init:
      init = False
    else:
      lastflashed = flashed
      flashed = flashed[1:]
      flashed += raise_neibs_en(g, lastflashed)
    for x in range(g.dim[0]):
      for y in range(g.dim[1]):
        coord = (x,y)
        if coord == (0,0):
          flashed = []
        if raise1(g, coord):
          log.debug(f"flashed {(x,y)}")
          flashed.append((x,y))
          flashs_num += 1
    break
    
def solve_d11pt1(ins):
  """Day 11 pt1 solution."""
  g = Grid(ins.splitlines())
  #g = g.iterate(ins)
  log.info(f"initial: {g}")
  log.info(g.show())
  log.debug(f" tns: {g.nb_tns}")
  raise_en(g)
  log.info(g.show())
  raise_en(g)
  log.info(g.show())
  return None

expected = 1656
n = 100
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
#result = solve_d11pt1(tests)
#aoc.assert_msg(f"test solve day 11 part 1; expected={expected}, found {result}", result == expected)
"Day 11 TBD" |> print

## Day 12: Passage Pathing

In [ ]:
"Day 12" |> print

tests = """
start-A
start-b
A-c
A-b
b-d
A-end
b-end
""".strip()

tests2 = """
dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc
""".strip()

In [ ]:
import networkx as nx  # Graph theory module
import matplotlib.pyplot as plt

def solve_d12pt1(ins):
  """Day 12 pt1 solution."""
  ins = ins.splitlines()
  g = nx.DiGraph()
  for ln in ins:
    #nd_st, nd_en = ln.split("-")
    nodes = ln.split("-")
    if "start" in nodes:
      other = nodes |> filter$(-> not _ == "start") |> list |> .[0]
      g.add_edge("start", other)
    elif "end" in nodes:
      other = nodes |> filter$(-> not _ == "end") |> list |> .[0]
      g.add_edge(other, "end")
    else:
      g.add_edge(nodes[0], nodes[1])
      g.add_edge(nodes[1], nodes[0])
  log.info(f"G:: nodes={g.nodes} edges={g.edges}")
  p = nx.shortest_path(g, source="start", target="end")
  log.info(f"G:: shortest_path={p}")
  ps = nx.all_simple_paths(g, source="start", target="end") |> list
  log.info(f"G:: all_simple_paths={ps}")
  log.info(f"G:: nodes_with_selfloops={list(nx.nodes_with_selfloops(g))}")
  log.info(f"G:: simple_cycles={list(nx.simple_cycles(g))}")
  #ps = nx.all_paths(g, source="A", target="A") |> list
  #log.info(f"G:: all_simple_paths[A->A]={ps}")
  #nx.draw_networkx(g)
  #nx.draw_networkx(g, pos=nx.circular_layout(g))
  #nx.draw_networkx(g, pos=nx.random_layout(g))
  #nx.draw_networkx(g, pos=nx.spiral_layout(g))
  #nx.draw_networkx(g, pos=nx.spring_layout(g)) # no edge overlaps !
  #nx.draw_networkx(g, pos=nx.planar_layout(g))
  #nx.draw_networkx(g, pos=nx.kamada_kawai_layout(g))
  nx.draw_networkx(g, pos=nx.spectral_layout(g))
  plt.show()
  return None

expected = 10
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
#result = solve_d12pt1(tests2)
#aoc.assert_msg(f"test solve day 8 part 1; expected={expected}, found {result}", result == expected)
"Day 12 TBD" |> print

## Day 13: Transparent Origami

In [ ]:
"Day 13" |> print

tests = """
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5
""".strip()

In [ ]:
def nda_repr(nda):
  s = str(nda).replace(" ","").replace("[","").replace("]","")
  return s.replace("0", ".").replace("1", "#")
  
def solve_d13pt1(ins, fold_count=None):
  """Day 13 pt1 solution."""
  backd = np.get_printoptions()
  np.set_printoptions(threshold=32_000, linewidth=32_000)
  ins, cmds = ins.split("\n\n")
  ins = ins.splitlines() |> map$(-> (_.split(",") |> map$(int) |> list) ) |> list
  cmds = cmds.splitlines() |> map$(-> (_.replace("fold along ", "")) ) \
    |> map$(-> (_.split("=")) ) |> list
  for cmd in cmds:
    cmd[1] = int(cmd[1])
  maxx = ins |> map$(-> _[0] ) |> list |> max$()
  maxy = ins |> map$(-> _[1] ) |> list |> max$()
  log.debug(f"ins={ins} max=[{maxx}, {maxy}] , cmds={cmds}")
  #nda = np.ndarray( shape=[maxx+1, maxy+1], dtype='i1' )
  nda = np.full(shape=[maxy+1, maxx+1], fill_value=0, dtype="i1")
  for cell in ins:
    nda[cell[1], cell[0]] = 1
  log.info("initial MAT\n" + nda_repr(nda))
  log.info("initial MAT\n" + str(nda))
  ct = -1
  for idx, cmd in enumerate(cmds):
    if fold_count is not None and idx >= fold_count:
      log.info(f"break after fold no {fold_count}")
      break
    dim, ofs = cmd
    log.info(f"fold {dim} on {ofs}")
    if dim == "y":
      a = nda[0:ofs,]
      b = nda[ofs+1:,]
      b = np.flipud(b) # flip(axis=0)
    elif dim == "x":
      a = nda[:,0:ofs]
      b = nda[:,ofs+1:]
      b = np.fliplr(b) # flip(axis=1)
    log.debug(f"? a.shape {a.shape} == {b.shape} b.shape ?")
    if a.shape[0] != b.shape[0]:
      m = max(a.shape[0], b.shape[0])
      log.info(f"reshape y max={m}")
      if a.shape[0] < m:
        a.resize([m, a.shape[1]])
      else:
        b.resize([m, b.shape[1]])
    elif a.shape[1] != b.shape[1]:
      log.info(f"reshape x max={m}")
      m = max(a.shape[1], b.shape[1])
      if a.shape[0] < m:
        a.resize([a.shape[0], m])
      else:
        b.resize([b.shape[0], m])
    log.debug(f"? a.shape {a.shape} == {b.shape} b.shape ?")
    assert(a.shape == b.shape)
    c = np.sign(a+b)
    nda = c
    #log.info("1\n" + nda_repr(a))
    #log.info("2\n" + nda_repr(b))
    #log.info("2t\n" + nda_repr(bb))
    ct = np.sum(c)
    log.info(f"folded {dim} on {ofs}; count={ct}; MAT=\n" + nda_repr(c))
  #np.set_printoptions(backd)
  np.set_printoptions(threshold=backd["threshold"], linewidth=backd["linewidth"])
  return ct

expected = 17
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
#result = solve_d13pt1(tests, fold_count=1)
#aoc.assert_msg(f"test solve day 13 part 1; expected={expected}, found {result}", result == expected)
"Day 13 TBD" |> print

In [ ]:
ins = aoc.read_file_to_str("./in/day13.in")
#out = solve_d13pt1(ins)
#f"day 13 part 1 output: {out}" |> print
"Day 13 TBD" |> print

## Day 14: Extended Polymerization

In [ ]:
"Day 14" |> print

tests = """
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
""".strip()

In [ ]:
def get_poly(ins, steps=1):
  """get the polymer chain after n steps"""
  ins, cmds = ins.split("\n\n")
  cmds = cmds.splitlines() |> map$(-> _.split(" -> ")) |> list
  log.trace(f"ins={ins} cmds={cmds}")
  d = {}
  for cmd in cmds:
    inps, outps = cmd
    outps = inps[0] + outps + inps[1]
    d[inps] = outps
  log.trace(f"  d={d}")
  for stepidx in range(steps):
    o = ""
    for idx in range(len(ins)-1):
      pr = ins[idx:idx+2]
      log.trace(f"  pair={pr}")
      if pr in d.keys():
        o += d[pr][0:2] # omit last input letter
      else:
        log.warn("pair {pr} not in recipes")
    o += ins[-1]
    log.trace(f"step#{stepidx+1} out={o}")
    ins = o
  return o

def solve_d14pt1(ins, steps=1):
  """Day 14 pt1 solution."""
  res = get_poly(ins, steps=steps)
  freqs = defaultdict(int)
  for c in res:
    freqs[c] += 1
  rc = max(freqs.values()) - min(freqs.values())
  log.info(f"rc={rc}, freqs={freqs}")
  return rc


expected = [
  [1, "NCNBCHB"],
  [2, "NBCCNBBBCBHCB"],
  [3, "NBBBCNCCNBBNBNBBCHBHHBCHB"],
  [4, "NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB"],
]
log.setLevel(logging.DEBUG)
#log.setLevel(logging.INFO)
for e in expected:
  n, expc = e[0], e[1]
  res = get_poly(tests, steps=n)
  aoc.assert_msg(f"test day 14 get_poly; n={n} expected={expc}, found {res}", res == expc)

# after step 5 it has len 97
# after step 10 it has len 3073
res = get_poly(tests, steps=5)
expc, reslen = 97, len( res )
aoc.assert_msg(f"test solve day 14 part 1; n={n} len={expc}, found {reslen}", reslen == expc)

res = get_poly(tests, steps=10)
expc, reslen = 3073, len( res )
aoc.assert_msg(f"test solve day 14 part 1; n={n} len={expc}, found {reslen}", reslen == expc)


n = 10
expc = 1588
rc = solve_d14pt1(tests, steps=10)
aoc.assert_msg(f"test solve day 14 part 1; n={n} len={expc}, found {rc}", rc == expc)

In [ ]:
ins = aoc.read_file_to_str("./in/day14.in")
out = solve_d14pt1(ins, steps=10)
f"day 14 part 1 output: {out}" |> print

In [ ]:
"Day 14 part 2" |> print

def get_polytns(ins, steps=1):
  """get the polymer chain after n steps"""
  ins, cmds = ins.split("\n\n")
  cmds = cmds.splitlines() |> map$(-> _.split(" -> ")) |> list
  log.trace(f"ins={ins} cmds={cmds}")
  cmdsd = {}
  for cmd in cmds:
    inps, outp = cmd
    outps = []
    outps.append(inps[0] + outp)
    outps.append(outp + inps[1])
    cmdsd[inps] = outps
  log.trace(f"  cmds={cmdsd}")
  pairs = defaultdict(int)
  for idx in range(len(ins)-1):
    pr = ins[idx:idx+2]
    pairs[pr] += 1
  log.trace(f"  pairs0={pairs}")
  for idx in range(steps):
    newd = pairs.copy()
    for inpair, ct in pairs.items():
      op1, op2 = cmdsd[inpair]
      log.trace(f"  tn: {inpair}:{ct} tn={[op1, op2]}")
      newd[inpair] -= ct
      newd[op1] += ct
      newd[op2] += ct
    pairs = newd
  return pairs

def solve_d14pt2(ins, steps=1):
  """Day 14 pt1 solution."""
  lastchar = ins.splitlines()[0][-1]
  res = get_polytns(ins, steps=steps)
  log.trace(f"get_polytns res={res} last-c={lastchar}")
  freqs = defaultdict(int)
  for pair, ct in res.items():
    p1, p2 = pair
    freqs[p1] += ct
    #freqs[p2] += ct # omit doubles
  freqs[lastchar] += 1 # fix frequencies, add last letter in chain
  log.trace(f"  freqs={freqs}")
  rc = max(freqs.values()) - min(freqs.values())
  return rc

rc = solve_d14pt2(tests, steps=0)

rc = solve_d14pt2(tests, steps=1)
#print("NCNBCHB")

rc = solve_d14pt2(tests, steps=2)
#print("NBCCNBBBCBHCB")

#rc = solve_d14pt2(tests, steps=2)

n = 10
expc = 1588
rc = solve_d14pt2(tests, steps=n)
aoc.assert_msg(f"test solve day 14 part 2; n={n} len={expc}, found {rc}", rc == expc)

rc = solve_d14pt2(ins, steps=10)
log.info(f"redo pt1={rc}")

out = solve_d14pt2(ins, steps=40)
f"day 14 part 2 output: {out}" |> print